# Dataset: *gurlitt*

- df.to_csv(r'D:\論文\PHEME9\Data\CSV\gurlitt-df.csv',index=False)
- src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\gurlitt-src.csv',index=False)

In [1]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\gurlitt-df.csv', encoding = 'utf-8', header = 0)
src_tw_df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\gurlitt-src.csv', encoding = 'utf-8', header =0)
print(df.shape)
print(src_tw_df.shape)

(164, 11)
(138, 7)


In [16]:
df.loc[23,['reply_tweet','cleaned_reply_tw','reply_sentiment']]

reply_tweet             NaN
cleaned_reply_tw        NaN
reply_sentiment     Neutral
Name: 23, dtype: object

In [21]:
df.loc[48,['reply_tweet','cleaned_reply_tw','reply_sentiment']]

reply_tweet             NaN
cleaned_reply_tw        NaN
reply_sentiment     Neutral
Name: 48, dtype: object

In [3]:
df.isnull().sum()

src_tweet_id          0
src_user_id           0
src_tweet             0
src_date              0
reply_tweet_id      123
reply_user_id       123
reply_tweet         123
reply_date          123
label                 0
cleaned_reply_tw    125
reply_sentiment       0
dtype: int64

In [16]:
df.dropna(inplace = True)
df.reset_index(drop=True,inplace=True)
df.shape

(39, 11)

In [4]:
src_tw_df.isnull().sum()

src_tweet_id      0
src_user_id       0
src_tweet         0
src_date          0
label             0
cleaned_src_tw    0
src_sentiment     0
dtype: int64

## Emotion Analysis

### roBERTa-base model
<https://huggingface.co/cardiffnlp/twitter-roberta-base-emotion>

In [5]:
import numpy as np
from scipy.special import softmax

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
ro_tokenizer = AutoTokenizer.from_pretrained(MODEL)
ro_model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [6]:
def get_emotion_roBERT(text):
    
    def preprocess(txt):
        new_text = []
        for t in txt.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)
    
    labels = ['anger','joy','optimism','sadness']
    text = preprocess(text)
    encoded_input = ro_tokenizer(text, return_tensors='pt')
    output = ro_model(**encoded_input)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    emo = labels[ranking[0]]
    return emo

In [8]:
emotion_ro = df.cleaned_reply_tw.astype(str).apply(get_emotion_roBERT)

In [9]:
df['reply_emo_ro'] = emotion_ro

In [10]:
df.to_csv(r'D:\論文\PHEME9\Data\CSV\gurlitt-df-2.csv',index=False)

In [11]:
src_emo_ro = src_tw_df.cleaned_src_tw.astype(str).apply(get_emotion_roBERT)
src_tw_df['src_emo_ro'] = src_emo_ro

In [12]:
src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\gurlitt-src-2.csv',index=False)

 - robert : 'anger','joy','optimism','sadness'
 - t5: 'sadness', 'joy', 'love', 'anger', 'fear', 'surprise' 
 - text2emotion: 'angry, 'fear', 'happy, 'sad', 'surprise'

In [15]:
print("source tweet:", df.src_tweet[18],'\n')
print("original reply tw: ", df.reply_tweet[18])
print("cleaned reply tw: ", df.cleaned_reply_tw[18])

print('roBerta: ', df.reply_emo_ro[18])

source tweet: ‘The Gurlitt collection should be sold to benefit Jewish organisations’ - The Art Newspaper http://t.co/DZLCSAoGIp 

original reply tw:  nan
cleaned reply tw:  nan
roBerta:  joy
